In [5]:
# Import everything that will be used
from keras.models import Sequential
from keras.layers import *
from keras.preprocessing.sequence import pad_sequences
import gensim
import sys
import numpy as np
import pandas as pd
import string
import logging
import random
import matplotlib.pyplot as plt

# Loading dataset
df = pd.read_csv('../data/cleaned_dataset.csv')
# Clean rows with nan text and drop all columns except text and category
# df = df.dropna(subset=['text']).iloc[:, [1, 3]]

# Create an iterator object that returns words properly formatted for Word2Vec training
class NewsIterator:
    def __init__(self, dataset):
        self.dataset = dataset

        # The text contains a stupid special character, hence the + '–'
        self.translator = str.maketrans('', '', string.punctuation + '–')

    def __iter__(self):
        for news in self.dataset.iloc[:, 0]:
            # Make all characters lower-case
            news = news.lower()
            for sentence in news.split('.')[:-1]:
                # Clean string of all punctuation
                sentence = sentence.translate(self.translator)

                words = [w for w in sentence.split(' ') if w != '']

                yield words
                
sentences = NewsIterator(df)

In [10]:
# Set up logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Train Word2Vec model with gensim on the dataset
w2v_model = gensim.models.Word2Vec(sentences, window=5, min_count=10, workers=8)

# Retrieve the weights from the model. This is used for initializing the weights
# in a Keras Embedding layer later
w2v_weights = w2v_model.wv.vectors
vocab_size, embedding_size = w2v_weights.shape

print("Vocabulary Size: {} - Embedding Dim: {}".format(vocab_size, embedding_size))

# Some validation on the quality of the Word2Vec model
print(w2v_model.wv.most_similar('1', topn=3))
print(w2v_model.wv.most_similar('janeiro', topn=3))
print(w2v_model.wv.most_similar('palmeiras', topn=3))
print(w2v_model.wv.most_similar('ufsc', topn=3))
print(w2v_model.wv.most_similar(positive=['rei', 'mulher'], negative=['homem'], topn=3))

def word2token(word):
    try:
        return w2v_model.wv.vocab[word].index
    # If word is not in index return 0. I realize this means that this
    # is the same as the word of index 0 (i.e. most frequent word), but 0s
    # will be padded later anyway by the embedding layer (which also
    # seems dirty but I couldn't find a better solution right now)
    except KeyError:
        return 0
def token2word(token):
    return w2v_model.wv.index2word[token]

AttributeError: 'float' object has no attribute 'lower'